In [2]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, MambaForCausalLM
from datasets import load_dataset
from evaluate import load

In [2]:
dataset = load_dataset("Trelis/tiny-shakespeare")

In [3]:
dataset.shape

{'train': (472, 1), 'test': (49, 1)}

In [4]:
dataset["train"]

Dataset({
    features: ['Text'],
    num_rows: 472
})

In [5]:
dataset["test"]["Text"][0]

"TRANIO:\nIs this your speeding? nay, then, good night our part!\n\nPETRUCHIO:\nBe patient, gentlemen; I choose her for myself:\nIf she and I be pleased, what's that to you?\n'Tis bargain'd 'twixt us twain, being alone,\nThat she shall still be curst in company.\nI tell you, 'tis incredible to believe\nHow much she loves me: O, the kindest Kate!\nShe hung about my neck; and kiss on kiss\nShe vied so fast, protesting oath on oath,\nThat in a twink she won me to her love.\nO, you are novices! 'tis a world to see,\nHow tame, when men and women are alone,\nA meacock wretch can make the curstest shrew.\nGive me thy hand, Kate: I will unto Venice,\nTo buy apparel 'gainst the wedding-day.\nProvide the feast, father, and bid the guests;\nI will be sure my Katharina shall be fine.\n\nBAPTISTA:\nI know not what to say: but give me your hands;\nGod send you joy, Petruchio! 'tis a match.\n\nGREMIO:\nAmen, say we: we will be witnesses.\n\nPETRUCHIO:\nFather, and wife, and gentlemen, adieu;\nI will 

In [6]:
len(dataset["test"]["Text"][0])

2859

In [7]:
# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"

In [8]:
model = MambaForCausalLM.from_pretrained("state-spaces/mamba-370m-hf").to(device)

/home/ivantllo/mamba_evaluation/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The fast path is not available because on of `(selective_state_update, selective_scan_fn, causal_conv1d_fn, causal_conv1d_update, mamba_inner_fn)` is None. Falling back to the naive implementation. To install follow https://github.com/state-spaces/mamba/#installation and https://github.com/Dao-AILab/causal-conv1d


In [9]:
tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-370m-hf")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
encodings = tokenizer(dataset["test"]["Text"][:2], return_tensors= "pt", padding=True)

In [11]:
model.config.get_config_dict("state-spaces/mamba-370m-hf")

({'architectures': ['MambaForCausalLM'],
  'bos_token_id': 0,
  'conv_kernel': 4,
  'd_inner': 160,
  'd_model': 1024,
  'dt_rank': 'auto',
  'eos_token_id': 0,
  'expand': 2,
  'fused_add_norm': True,
  'hidden_act': 'silu',
  'hidden_size': 1024,
  'initializer_range': 0.1,
  'intermediate_size': 2048,
  'layer_norm_epsilon': 1e-05,
  'model_type': 'mamba',
  'n_layer': 48,
  'num_hidden_layers': 48,
  'pad_token_id': 0,
  'pad_vocab_size_multiple': 8,
  'rescale_prenorm_residual': False,
  'residual_in_fp32': True,
  'rms_norm': True,
  'ssm_cfg': {},
  'state_size': 16,
  'time_step_floor': 0.0001,
  'time_step_init_scheme': 'random',
  'time_step_max': 0.1,
  'time_step_min': 0.001,
  'time_step_rank': 64,
  'time_step_scale': 1.0,
  'torch_dtype': 'float32',
  'transformers_version': '4.39.0.dev0',
  'use_bias': False,
  'use_cache': True,
  'use_conv_bias': True,
  'vocab_size': 50280,
  '_commit_hash': 'b519127f5bfaaa1c27dd938dad051ec360972b23'},
 {})

In [ ]:
perplexity = load("perplexity", module_type="metric")

In [ ]:
input_ids = encodings.input_ids.to(device)
outputs = model(input_ids)

In [ ]:
outputs

In [ ]:
results = perplexity.compute(predictions=predictions, model_id='gpt2', add_start_token=False)

In [12]:
def calculate_perplexity(max_length, stride, seq_len):
    nlls = []
    prev_end_loc = 0
    for begin_loc in range(0, seq_len, stride):
        end_loc = min(begin_loc + max_length, seq_len)
        trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
        input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100

        with torch.no_grad():
            outputs = model(input_ids, labels=target_ids)

        # loss is calculated using CrossEntropyLoss which averages over valid labels
        # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
        # to the left by 1.
            neg_log_likelihood = outputs.loss

        nlls.append(neg_log_likelihood)

        prev_end_loc = end_loc
        if end_loc == seq_len:
            break
    return nlls

In [13]:
# max_length = model.config.n_positions
max_length = 4000  # ¿¿ese es el context length?? parece que no existe esta limitación para este modelo, sino que su rendimiento empeora con el largo de la secuencia y esto lo define la longitud del dataset de entrenamiento
seq_len = encodings.input_ids.size(1)

In [14]:
stride = 512
nlls = calculate_perplexity(max_length, stride, seq_len)

ppl = torch.exp(torch.stack(nlls).mean())
print(f"Perplexity: {ppl.item()}")

Perplexity: 24.20311164855957


In [15]:
stride = 2
nlls = calculate_perplexity(max_length, stride, seq_len)

ppl = torch.exp(torch.stack(nlls).mean())
print(f"Perplexity: {ppl.item()}")

Perplexity: 24.20311164855957
